# **Gen AI Model Mock-up: Helping Users Leave Higher Quality Reviews**

# Nivedita Prasad

# Introduction to Machine Learning Model flow


High-Level Steps:

1. Preprocessing:

The user’s review is preprocessed (e.g., lowercasing, punctuation fixing) to ensure consistent handling during further steps.

2. Sentiment Analysis:

The overall sentiment of the review is analysed using [VADER](https://medium.com/@rslavanyageetha/vader-a-comprehensive-guide-to-sentiment-analysis-in-python-c4f1868b0d2e).

Based on the compound score (between -1 and 1):

* If the sentiment is positive, the model says, "It sounds like you liked our product!"

* If negative, it says, "It sounds like you had concerns with the product."

* For neutral/mixed, it provides, "It seems like you had mixed feelings about the product."

3. Dynamic Suggestions using OpenAI's GPT model:

* GPT-4 generates suggestions based on the user’s review. For illustration purposes, we focus on how GPT will help users review their skincare products in more detail.
* If the review discusses 'redness of skin', GPT-4 might suggest mentioning texture or absorption.
* We use the OpenAI API key here, and feed samples of publically availably review data

4. Final text generation 'Review Suggestions':

* Finally, the function combines the sentiment-based feedback with the dynamic suggestions and generates enhanced text for their reviews, based on their sentiment


In [2]:
#Importing libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
import re

# 1. Load the model and tokeniser

A key problem we're addressing here, is what if the user is really short for time so inputs a bunch of gibberish? Or they think saying 'good' or 'fine' suffices - but actually, other users really want to understand the specifics of this, to inform their purchasing decision.

We want to enhance prompt engineering by preprocessing any input text:

# 2. Preprocess the input text

We can define a function for this:

In [3]:
def preprocess(text):
    """
    Purpose:
    --------
    Preprocesses review input text by trying to address the following:
    - A string of words pushed together with no spaces (e.g., "okaynotbad" → "okay, not bad").
    - Misspellings which make it hard for other users to derive meaning
    - Removing gibberish and placehorder words (e.g., 'xxxxxx') where a user might feel lazy to expand
    - Punctuation checks

    Inputs:
    -------
    text : str
        The review text input by the Amazon user

    Outputs:
    --------
    str
        The cleaned and preprocessed version of the input text, suitable for use in text generation.
    """
    # Lowercase the text
    text = text.lower()

    # Split words using regex
    text = re.sub(r'([a-z])([A-Z])', r'\1, \2', text)

    # Common punctuation problems
    text = re.sub(r'[,]+', ', ', text)  # Fix multiple commas
    text = re.sub(r'[ ]{2,}', ' ', text)  # Remove extra spaces

    return text.strip()

# 3. Sentiment Analysis

There are a lot of cool packages we can also harness to consider sentiment analysis building on our preprocessing. See below:

https://www.bairesdev.com/blog/best-python-sentiment-analysis-libraries/

For this industry project, let's try out 'Vader' from NLTK:

In [4]:
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## We will keep in the below function for now in case we want to build on our prompts using the postivie and negative features we've extracted - however, for this demo, we care more about the sentiment score.

In [5]:
# Initialize the sentiment analyzer
sentiment = SentimentIntensityAnalyzer()

def analyse_sentiment(text):
    """
    Purpose:
    --------
    Performs sentiment analysis of tokens in input text using Vader
    It classifies each part as positive, negative, or neutral based on its polarity score.

    Inputs:
    -------
    text : str
        The input review text

    Outputs:
    --------
    positive_features : str
        A string containing positive features.

    negative_features : str
        A string containing negative features.
    """
    words = text.split()  # Split text into individual words (or could be phrases)

    #store as lists

    positive_features = []
    negative_features = []

    #Sentiment analysis using vader

    for word in words:
        # Get sentiment scores
        sentiment_score = sentiment.polarity_scores(word)['compound']

        # Classify as positive, negative, or neutral
        if sentiment_score > 0.05:  # Threshold for positive sentiment
            positive_features.append(word)
        elif sentiment_score < -0.05:  # Threshold for negative sentiment
            negative_features.append(word)

    # Join the features into strings
    positive_str = ' '.join(positive_features)
    negative_str = ' '.join(negative_features)

    return positive_str, negative_str

# 3. Prompt generation to help organise reviews better

Once we get that preprocessed input text, we need to think about how we can actually create an informative sturctured prompt for our prompt generation. The below is just one initial way of doing so, but of course requires more finetuning:

In [7]:
#we need to import libraries
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

In [ ]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


The below loads in the API key that you should have loaded as part of your environment when loading the script:

In [15]:
# Import necessary libraries
import os
from dotenv import load_dotenv

# Load environment variables from .env file (if it exists)
load_dotenv()

# Get the API key from the environment variables
api_key = os.getenv('API_KEY')

# Check if the API key exists
if not api_key:
    raise ValueError("API Key not found. Please set the API_KEY environment variable or add it to a .env file.")

# Now you can use 'api_key' in your code where needed


ValueError: API Key not found. Please set the API_KEY environment variable or add it to a .env file.

In [9]:
from openai import OpenAI

# Set your OpenAI API key here
client = OpenAI(api_key = 'api_key')

def generate_prompt(input_text, template_type="basic", length_threshold=50):
    """
    Constructs a prompt for a personal care or health product based on user input,
    refined by GPT-3 or GPT-4 to generate more natural language suggestions.
    GPT is used to generate suggestions based on the input, assuming it's related to skincare products.
    """

    # Preprocess the input text
    input_text = preprocess(input_text)

    # Determine the length of the input text
    input_length = len(input_text.split())

    # Get overall sentiment score using VADER
    sentiment_score = sentiment.polarity_scores(input_text)['compound']

    # Provide dynamic feedback based on sentiment score
    if sentiment_score >= 0.05:
        sentiment_prompt = "It sounds like you liked our product!"
    elif sentiment_score <= -0.05:
        sentiment_prompt = "It sounds like you had concerns with the product."
    else:
        sentiment_prompt = "It seems like you had mixed feelings about the product."

    # Use GPT-4 to generate suggestions based on input text
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an expert on skin care products and want improve the quality of users' reviews by offering suggestions - you don't need to respond to this in your generated prompt though."},
            {"role": "user", "content": f"Given the following review of a skincare product: {input_text}, generate refined sentence starters with placeholders to describe how the user felt or what they thought of the product in more detail. The sentence starters should encourage the reviewer to add details without introducing new content, like 'The product felt...and after using it for..., I noticed....' For example, you could mention value for money with something like, 'For the price, I found the product to be..., and it lasted me... .'. Always provide the refined sentences in this format, with some options for synonyms to fill in the placeholders e.g., (high,low; costly, affordable). If the placeholder is an adjective, we should also provide some options for what words. Please provide the response as a JSON object containing an array of the bullet points and the initial sentence as a key value pair inside the JSON object. Only return a JSON object."}
        ],
        max_tokens=500,
        n=1,
        stop=None,
        temperature=0.7,
    )

    # Extract the suggestion generated by GPT-3/4
    gpt_suggestion = response.choices[0].message.content.strip()


    # Construct the final output for reviews
    final_prompt = f"{sentiment_prompt} We’ve generated some ideas to help you add more detail to your review. Take a look at these suggested updates:\n\n{gpt_suggestion}"

    return final_prompt


# 4. Test with input review data!

**At this stage, we are returning generated 'sentence starters' as a JSON object for the engineers to retrieve as part of a larger application.**

In [10]:
response = generate_prompt("lovely product. blends well")

In [11]:
print(response)

It sounds like you liked our product! We’ve generated some ideas to help you add more detail to your review. Take a look at these suggested updates:

```json
{
  "initial_sentence": "lovely product. blends well",
  "refined_sentence_starters": [
    "The product felt (smooth, luxurious, light) on my skin, and after using it for (a week, two weeks, a month), I noticed (a significant improvement, a subtle change, no difference) in my skin's texture.",
    "For the price, I found the product to be (high, low; costly, affordable), and it lasted me (a long time, just a few weeks, as expected).",
    "I particularly appreciated how the product (absorbed, blended, felt) on my skin, making my daily routine (easier, more enjoyable, quicker).",
    "After using this product consistently, my skin felt (hydrated, smoother, healthier) and looked (brighter, clearer, more even-toned).",
    "Compared to other products I've tried, this one (stands out, is similar, falls short) because of its (texture,

In [12]:
response3 = generate_prompt("There are 12 individually wrapped bath bombs in this set and they all smell the same to me. The box is wrapped around the center in packaging tape which looks hideous. The bombs themselves, however, are presentation quality in the sense that they are wrapped in tissue paper and then also shrink wrapped in plastic under that.<br /><br />I hardly detected any fragrance with these and they all smell the same to me. Since I personally do not like the color rings caused by bath bombs in my bath, I test them in a foot bath first to either bathe my hands or my feet. Because I was skeptical of these, I bathed my hands.<br /><br />One bath bomb dissolves in about three minutes in close to hot water and the suds last about 10-15 minutes. The fragrance is almost non-existent and the results are horrendous. First of all, as I was filling the foot bath, the color of the bombs sprayed all over my sink and kitchen backsplash just as a result of running water over the bomb")

In [13]:
print(response3)

It sounds like you had concerns with the product. We’ve generated some ideas to help you add more detail to your review. Take a look at these suggested updates:

```json
{
    "initial_sentence": "There are 12 individually wrapped bath bombs in this set and they all smell the same to me.",
    "refined_sentence_starters": [
        "The box is wrapped around the center in packaging tape which looks (unattractive, unappealing, hideous).",
        "The bombs themselves are presentation quality in the sense that they are wrapped in tissue paper and then also shrink wrapped in plastic under that.",
        "I hardly detected any fragrance with these and they all smell the same to me.",
        "Since I personally do not like the color rings caused by bath bombs in my bath, I test them in a foot bath first to either bathe my hands or my feet.",
        "Because I was skeptical of these, I bathed my hands.",
        "One bath bomb dissolves in about three minutes in (hot, warm) water and the

# 5. What are the caveats and next steps?

* No fine-tuning: Right now, we’re relying on GPT-4 as a general-purpose language model without training on more specific Amazon reviews data.

* VADER for sentiment analysis, which works well for general text but can struggle to capture more complex feelings about a product

